In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import os
import sys
import gc
import json
import configparser
from datetime import datetime
import time

from utils.f2thresholdfinder import *
from utils.loaderjpg import *
from utils.generator import *
from utils.custommetrics import *
from utils.predictorjpg import *
from utils.file import *

from pretrained.custommodels import *
from pretrained.vgg19 import vgg19_model_custom_top_hack

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1060 6GB (0000:01:00.0)


In [3]:
labels = ['slash_burn', 'clear', 'blooming', 'primary', 'cloudy', 'conventional_mine', 'water', 'haze', 'cultivation', 'partly_cloudy', 'artisinal_mine', 'habitation', 'bare_ground', 'blow_down', 'agriculture', 'road', 'selective_logging']

file_uuid = time.strftime("%Y%m%d-%H%M%S")

data_dir = 'D:/Downloads/amazon/'

rescaled_dim = 224

# model_name = 'densenet121'
# should_load_model = False
# model_filepath = None
# weights_filepath = 'D:/Downloads/amazon/bottleneck/densenet121/frozen73_20170716-171641_weights_only.h5'

# model_name = 'resnet50'
# should_load_model = False
# model_filepath = None
# weights_filepath = 'D:/Downloads/amazon/bottleneck/resnet50/frozen38_20170717-092206_weights_only.h5'

# model_name = 'vgg16'
# should_load_model = True
# model_filepath = 'D:/Downloads/amazon/bottleneck/vgg16/frozen11_20170706-011852.h5'
# weights_filepath = None

model_name = 'vgg19'
should_load_model = False
model_filepath = 'D:/Downloads/amazon/bottleneck/vgg19/frozen13_20170718-183101.h5' # Bug with Keras
weights_filepath = 'D:/Downloads/amazon/bottleneck/vgg19/frozen13_20170718-183101_weights_only.h5'


if should_load_model:
    print('loading model file: {}'.format(model_filepath))
    model = load_model(model_filepath)
else:
    print('loading model weights file: {}'.format(weights_filepath))
    model = custom_top_model(model_name, num_classes=17, num_frozen_layers=0)
    model.load_weights(weights_filepath)


ValueError: You are trying to load a weight file containing 17 layers into a model with 19 layers.

In [ ]:
df_train = pd.read_csv(data_dir + 'train_v2.csv')
x_train, y_train = load_training_set(df_train, rescaled_dim)
print(x_train.shape)
print(y_train.shape)

In [ ]:
number_of_samples = x_train.shape[0]
split = int(number_of_samples * 0.90)
                     
x_valid, y_valid = x_train[split:], y_train[split:]

del x_train
del y_train

number_validations = number_of_samples - split

In [ ]:
img_normalization = image_normalization_func(model_name)
valid_datagen = BottleNeckImgGenerator(normalization=img_normalization)

In [ ]:
y_predictions, optimized_thresholds = predict_with_optimal_thresholds(x_valid, y_valid, valid_datagen, model)

threshold_df = pd.DataFrame({'label':labels, 
                             'optimized_threshold':optimized_thresholds})
print(threshold_df)

In [7]:
sample_submission_filepath = data_dir + 'sample_submission_v2.csv'
real_submission_filepath = data_dir + 'my_submissions/tta_submission_{}_{}.csv'.format(model_name, file_uuid)

img_normalization = image_normalization_func(model_name)
test_datagen = BottleNeckImgGenerator(normalization=img_normalization)

In [8]:
make_submission(model,
                optimized_thresholds,
                rescaled_dim, 
                labels,
                sample_submission_filepath,
                real_submission_filepath,
                test_datagen)

KeyboardInterrupt: 